# Definition of used classes

In [1]:
import os
import math
from datetime import datetime
from abc import ABC, abstractmethod
from typing import Any, Dict, List, Tuple
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow

from IPython.display import clear_output
from IPython.display import Video

from pygame import Vector2 as Vec2
from pygame import Color as Color

# Constants
FPS: int = 30
SCREEN_SIZE: Vec2 = Vec2(640, 440)
PITCH_SIZE: Vec2 = Vec2(600, 400)
PITCH_POS: Vec2 = (SCREEN_SIZE - PITCH_SIZE) / 2
POST_TOP_Y: float = PITCH_POS.y + PITCH_SIZE.y * 6 / 16
POST_BOT_Y: float = PITCH_POS.y + PITCH_SIZE.y * 10 / 16
LINE_WIDTH: int = 2

# Color palette
COLOR_WHITE: Color = Color(255, 255, 255)
COLOR_BLACK: Color = Color(0, 0, 0)
COLOR_LINE: Color = Color(174, 202, 137)
COLOR_PITCH: Color = Color(113, 152, 63)
COLOR_BACKGROUND: Color = Color(91, 143, 164)


class Drawable(ABC):
    @abstractmethod
    def draw(self, img):
        raise NotImplementedError()


class CirclePhysical(ABC):
    COUNT: int = 0

    def __init__(self, radius: float, mass: float, pos: Vec2, hooked: bool = False):
        self.id: int = CirclePhysical.COUNT
        CirclePhysical.COUNT += 1

        self.radius: float = radius
        self.mass: float = mass
        self.friction: float = 0.7 * mass
        self.v_max = 12 / math.pow(mass, 2 / 3)
        self.pos: Vec2 = pos
        self.vel: Vec2 = Vec2(0, 0)
        self.hooked: bool = hooked

    def setPos(self, pos: Vec2):
        self.pos = pos

    def setVel(self, vel: Vec2):
        self.vel = vel * self.mass

    def addVel(self, vel: Vec2):
        self.vel += vel

    def update(self, dt: float):
        self.pos += self.vel * dt

        self.vel += -self.vel * self.friction * dt
        self.pos += self.vel * dt

        if self.vel.magnitude() > self.v_max:
            self.vel = self.vel.normalize() * self.v_max

    @staticmethod
    def sphereCollisionVelocities(
            v1: Vec2, v2: Vec2,
            m1: float, m2: float,
            x1: Vec2, x2: Vec2) -> Tuple[Vec2, Vec2]:
        # Calculate new velocities after collision
        mass: float = 2 * m1 / (m1 + m2)
        v1_new: Vec2 = v1 - (mass * (v1 - v2).dot(x1 - x2) / pow((x1 - x2).length(), 2)) * (x1 - x2)
        v2_new: Vec2 = v2 - (mass * (v2 - v1).dot(x2 - x1) / pow((x2 - x1).length(), 2)) * (x2 - x1)
        return v1_new, v2_new

    def collide(self, other: 'CirclePhysical'):
        dist = self.pos.distance_to(other.pos)
        if dist < self.radius + other.radius:
            if dist == 0:
                self.pos += -self.vel.normalize() * 0.01

            self.vel, other.vel = CirclePhysical.sphereCollisionVelocities(
                self.vel, other.vel, self.mass, other.mass, self.pos, other.pos)

            if not self.hooked:
                self.pos += (self.pos - other.pos).normalize() * (self.radius + other.radius - dist) / 2
            else:
                self.vel = Vec2(0, 0)

            if not other.hooked:
                other.pos += (other.pos - self.pos).normalize() * (self.radius + other.radius - dist) / 2
            else:
                other.vel = Vec2(0, 0)


class Ball(CirclePhysical, Drawable):
    COLOR: Color = Color(255, 255, 255)
    MASS: float = 0.2
    RADIUS: float = 7

    def __init__(self, pos: Vec2 = Vec2(SCREEN_SIZE.x / 2, SCREEN_SIZE.y / 2)):
        super().__init__(
            radius=self.RADIUS, mass=self.MASS, pos=pos)

    def draw(self, img):
        cv2.circle(img, (int(self.pos.x), int(self.pos.y)), self.radius, self.COLOR, -1)


class Player(CirclePhysical, Drawable):
    MASS: float = 0.5
    RADIUS: float = 10

    def __init__(self, pos: Vec2, color: Color):
        super().__init__(
            radius=self.RADIUS, mass=self.MASS, pos=pos)

        self.color: Color = color

    def draw(self, img):
        cv2.circle(img, (int(self.pos.x), int(self.pos.y)), self.radius, self.color, -1)


class Post(CirclePhysical, Drawable):
    MASS: float = 1
    RADIUS: float = 6

    def __init__(self, pos: Vec2, color: Color):
        super().__init__(
            radius=self.RADIUS, mass=self.MASS, pos=pos, hooked=True)

        self.color: Color = color

    def draw(self, img):
        cv2.circle(img, (int(self.pos.x), int(self.pos.y)), self.radius, self.color, -1)


class Goal(Drawable):
    def __init__(self, dir: int, linePos: float, postTopY: float, postBotY: float, color: Color):
        self.dir: int = dir
        self.linePos: float = linePos
        self.yTop: float = postTopY
        self.yBot: float = postBotY
        self.color: Color = color

        self.postTop: Post = Post(Vec2(linePos, postTopY), color)
        self.postBot: Post = Post(Vec2(linePos, postBotY), color)

    def draw(self, img):
        cv2.line(img, (int(self.linePos), int(self.yTop)), (int(self.linePos), int(self.yBot)), self.color, LINE_WIDTH)
        self.postTop.draw(img)
        self.postBot.draw(img)


class Team(Drawable):
    def __init__(self, dir: int, color: Color):
        self.color: Color = color

        # Dir is 1 or -1
        # 1 is attacking from left to right
        # -1 is attacking from right to left
        self.dir: int = dir

        # Normalized dir is 0 or 1
        # 0 is attacking from left to right
        # 1 is attacking from right to left
        self.normalizedDir: int = -(dir - 1) / 2

        # Goal line for dir 1 is pitchPos.x
        # Goal line for dir -1 is pitchPos.x + pitchSize.x
        goalLineX: float = PITCH_POS.x + PITCH_SIZE.x * self.normalizedDir

        self.goal: Goal = Goal(dir, goalLineX, POST_TOP_Y, POST_BOT_Y, color)
        self.player: Player = Player(Vec2(goalLineX, SCREEN_SIZE.y / 2), color)

        self.score: int = 0

    def draw(self, img):
        self.goal.draw(img)
        self.player.draw(img)

    def reset(self):
        self.score = 0
        self.player.setPos(
            Vec2(self.goal.linePos + self.dir * PITCH_SIZE.x / 4, SCREEN_SIZE.y / 2 + self.dir * SCREEN_SIZE.y / 6))
        self.player.setVel(Vec2(0, 0))

    def applyAction(self, action: Vec2):
        # Convert action to direction and invert it in relation to team dir
        if action.length() != 0:
            vel: Vec2 = action.normalize() * self.dir

            # Apply velocity to player
            self.player.addVel(vel)


class Pitch(Drawable):
    def __init__(self):
        self.team1: Team = Team(1, Color(255, 0, 0))
        self.team2: Team = Team(-1, Color(0, 0, 255))

    def draw(self, img):
        # Draw pitch as a solid color filled rectangle with a line border
        cv2.rectangle(img, (int(PITCH_POS.x), int(PITCH_POS.y)),
                      (int(PITCH_POS.x + PITCH_SIZE.x), int(PITCH_POS.y + PITCH_SIZE.y)), COLOR_PITCH, -1)
        cv2.rectangle(img, (int(PITCH_POS.x), int(PITCH_POS.y)),
                      (int(PITCH_POS.x + PITCH_SIZE.x), int(PITCH_POS.y + PITCH_SIZE.y)), COLOR_LINE, LINE_WIDTH)

        # Draw center line
        cv2.line(img, (int(PITCH_POS.x + PITCH_SIZE.x / 2), int(PITCH_POS.y)),
                 (int(PITCH_POS.x + PITCH_SIZE.x / 2), int(PITCH_POS.y + PITCH_SIZE.y)), COLOR_LINE, LINE_WIDTH)
        cv2.circle(img, (int(PITCH_POS.x + PITCH_SIZE.x / 2), int(PITCH_POS.y + PITCH_SIZE.y / 2)),
                   int(PITCH_SIZE.y / 6), COLOR_LINE, LINE_WIDTH)

        # Draw goals
        self.team1.draw(img)
        self.team2.draw(img)

    def pitchCollision(self, sphere: CirclePhysical, dt: float) -> int:
        # Check sphere's collision with pitch
        # Return 0 if nothing happens
        # Return 1 if team sphere is in goal of team 1
        # Return 2 if team sphere is in goal of team 2

        # Check if the sphere collides with the pitch's top or bottom
        if (sphere.pos.y - sphere.radius < PITCH_POS.y or
                sphere.pos.y + sphere.radius > PITCH_POS.y + PITCH_SIZE.y):
            sphere.pos.y += -sphere.vel.y * dt
            sphere.vel.y *= -1

        # Check if the sphere collides with the goal
        if (sphere.pos.y > POST_TOP_Y and sphere.pos.y < POST_BOT_Y) and isinstance(sphere, Ball):
            # Check if the sphere collides with the goal of team 1
            if (sphere.pos.x + sphere.radius < self.team1.goal.linePos):
                return 1

            # Check if the sphere collides with the goal of team 2
            if (sphere.pos.x - sphere.radius > self.team2.goal.linePos):
                return 2

            return 0

        # Check if the sphere collides with the pitch's left or right
        if (sphere.pos.x - sphere.radius < PITCH_POS.x or
                sphere.pos.x + sphere.radius > PITCH_POS.x + PITCH_SIZE.x):
            sphere.pos.x += -sphere.vel.x * dt
            sphere.vel.x *= -1

        return 0


class ReplayBuffer():
    def __init__(self):
        self.buffer: List[Tuple(Vec2, Vec2, Vec2)] = []

    def snapshot(self, ball: Ball, team1: Team, team2: Team):
        self.buffer.append((ball.pos.copy(), team1.player.pos.copy(), team2.player.pos.copy()))

    def reset(self):
        self.buffer = []


class Haxball():
    def __init__(self):
        self.ball: Ball = Ball()
        self.pitch: Pitch = Pitch()
        self.player1: Player = self.pitch.team1.player
        self.player2: Player = self.pitch.team2.player

        self.circlePhysicals: List[CirclePhysical] = [
            self.ball, self.player1, self.player2,
            self.pitch.team1.goal.postTop, self.pitch.team1.goal.postBot,
            self.pitch.team2.goal.postTop, self.pitch.team2.goal.postBot
        ]

        self.paused: bool
        self.lastAction1: Vec2 = Vec2(0, 0)
        self.lastAction2: Vec2 = Vec2(0, 0)

        self.replayBuffer: ReplayBuffer = ReplayBuffer()

        self.reset()

    def reset(self):
        self.paused = False
        self.lastAction1 = Vec2(0, 0)
        self.lastAction2 = Vec2(0, 0)

        # Reset ball
        self.ball.setPos(Vec2(SCREEN_SIZE.x / 2, SCREEN_SIZE.y / 2))
        self.ball.setVel(Vec2(0, 0))

        # Reset teams
        self.pitch.team1.reset()
        self.pitch.team2.reset()

        # Save first snapshot to replay buffer
        self.replayBuffer.reset()
        self.replayBuffer.snapshot(self.ball, self.pitch.team1, self.pitch.team2)

    @staticmethod
    def getRelativePos(pos: Vec2, dir: int) -> Vec2:
        # Return player position normalized to the given dir
        # (0, PITCH_SIZE.y / 2) is the center of own goal line

        normalizedDir = -(dir - 1) / 2

        return (pos - SCREEN_SIZE * normalizedDir) * dir

    @staticmethod
    def getRelativeVel(vel: Vec2, dir) -> Vec2:
        # Return player velocity normalized to the given dir

        return vel * dir

    def info(self):
        info = dict()
        info["pitch_size"] = (int(PITCH_SIZE.x), int(PITCH_SIZE.y))
        info["pitch_pos"] = (int(PITCH_POS.x), int(PITCH_POS.y))
        info["screen_size"] = (int(SCREEN_SIZE.x), int(SCREEN_SIZE.y))
        info["player_radius"] = Player.RADIUS
        info["ball_radius"] = Ball.RADIUS
        info["post_radius"] = Post.RADIUS
        info["goal_width"] = POST_BOT_Y - POST_TOP_Y
        info["own_postTop_pos"] = self.getRelativePos(self.pitch.team1.goal.postTop.pos, 1)
        info["own_postBot_pos"] = self.getRelativePos(self.pitch.team1.goal.postBot.pos, 1)
        info["opp_postTop_pos"] = self.getRelativePos(self.pitch.team2.goal.postTop.pos, 1)
        info["opp_postBot_pos"] = self.getRelativePos(self.pitch.team2.goal.postBot.pos, 1)
        info["state_space"] = 12
        info["action_space"] = 2
        return info

    def state(self):
        return self.getState(1), self.getState(2), self.paused

    def render(self):
        img = np.zeros((int(SCREEN_SIZE.y), int(SCREEN_SIZE.x), 3), np.uint8)

        cv2.rectangle(img, (0, 0), (int(SCREEN_SIZE.x), int(SCREEN_SIZE.y)), COLOR_BACKGROUND, -1)

        self.pitch.draw(img)
        self.ball.draw(img)

        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        return img

    def showReplay(self, custom_string="", with_show_video=True):
        video_path = f"tmp_{custom_string}_{datetime.now().strftime('%d-%m-%y--%H-%M-%S')}.mp4"

        video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*"mp4v"), FPS, (self.render().shape[1::-1]))

        # Create video for whole replay buffer
        for snapshot in self.replayBuffer.buffer:
            # print(snapshot)
            self.ball.pos = snapshot[0]
            self.pitch.team1.player.pos = snapshot[1]
            self.pitch.team2.player.pos = snapshot[2]

            video.write(self.render())

        video.release()
        if with_show_video:
            fileToDisplay = "video.mp4"
            os.system(
                f"ffmpeg -loglevel level+fatal -y -i {video_path} -vcodec libx264 -x264opts keyint=123:min-keyint=120 -an {fileToDisplay}")
            display(Video(fileToDisplay, embed=True))
        else:
            print(video_path)

    def getState(self, team: int):
        dir: int
        reward: int
        lastAction: Vec2
        player: Player
        opponent: Player

        if team == 1:
            dir = 1
            reward = self.pitch.team1.score
            lastAction = self.lastAction1
            player = self.player1
            opponent = self.player2
        elif team == 2:
            dir = -1
            reward = self.pitch.team2.score
            lastAction = self.lastAction2
            player = self.player2
            opponent = self.player1
        else:
            raise Exception("Invalid team")

        ballPos: Vec2 = self.getRelativePos(self.ball.pos, dir)
        ballVel: Vec2 = self.getRelativeVel(self.ball.vel, dir)
        playerPos: Vec2 = self.getRelativePos(player.pos, dir)
        playerVel: Vec2 = self.getRelativeVel(player.vel, dir)
        opponentPos: Vec2 = self.getRelativePos(opponent.pos, dir)
        opponentVel: Vec2 = self.getRelativeVel(opponent.vel, dir)

        state: List[float] = [
            ballPos.x, ballPos.y, ballVel.x, ballVel.y,
            playerPos.x, playerPos.y, playerVel.x, playerVel.y,
            opponentPos.x, opponentPos.y, opponentVel.x, opponentVel.y
        ]

        return state, reward, lastAction

    def collide(self, sphere: CirclePhysical, dt: float) -> int:
        # Collide with other spheres
        for other in self.circlePhysicals:
            if other != sphere:
                sphere.collide(other)

        # Collide with pitch
        return self.pitch.pitchCollision(sphere, dt)

    def step(self, action1: Vec2, action2: Vec2):
        if self.paused:
            self.replayBuffer.snapshot(self.ball, self.pitch.team1, self.pitch.team2)
            return self.state()

        dt: float = 15 / FPS

        # Apply actions
        self.pitch.team1.applyAction(action1)
        self.pitch.team2.applyAction(action2)

        # Update players
        self.player1.update(dt)
        self.player2.update(dt)
        # Update ball
        self.ball.update(dt)

        # Check for collisions
        self.collide(self.player1, dt)
        self.collide(self.player2, dt)
        res: int = self.collide(self.ball, dt)

        # Save snapshot to replay buffer
        self.replayBuffer.snapshot(self.ball, self.pitch.team1, self.pitch.team2)

        self.lastAction1 = action1
        self.lastAction2 = action2

        # Check if a goal was scored
        if res == 1:
            # print("GOAL! for team 2")
            self.pitch.team2.score += 1
            self.paused = True
        elif res == 2:
            self.pitch.team1.score += 1
            # print("GOAL! for team 1")
            self.paused = True

        return self.state()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# The environment
The environment was defined in the Haxball class.<br>

### State
State is returned by function `step(action1, action2)` or `state()`.
State consists of 3 values: `state1, state2, done`:
* `state1` -- `Tuple(state, reward, lastAction)`:
  * `state` -- `Dict`: game state from the perspective of a player 1:
    * `ballPos` -- `Vec2`: position of the ball
    * `ballVel` -- `Vec2`: velocity of the ball
    * `playerPos` -- `Vec2`: position of the player
    * `playerVel` -- `Vec2`: velocity of the player
    * `opponentPos` -- `Vec2`: position of the opponent
    * `opponentVel` -- `Vec2`: velocity of the opponent
  * `reward` -- `int`: reward for the last action
  * `lastAction` -- `Vec2`: last action taken by the player
* `state2` -- same as `state1` but from the perspective of a player 2
* `done` -- `bool`: True if the game is over (goal was scored)


### Info
Function `info()` returns a dictionary containing information about the environment:
 * `pitch_size` -- size of the pitch
 * `pitch_pos` -- position of the top left corner of the screen
 * `screen_size` -- size of the whole screen (pitch + border)
 * `player_radius` -- radius of the player
 * `ball_radius` -- radius of the ball
 * `post_radius` -- radius of the post
 * `goal_width` -- width of the goal (y of the bottom post - y of the top post)

### Action
Action is continuous and is a `Vec2` type. It specifies the direction of the movement. It doesn't have to be normalized (magnitude doesn't affect the velocity).

Action is relative to the player's goal. `Vec(1, 0)` (forward movement along x-axis) for one player means going to the right side of the screen, but for another it means going to the left (from the viewer's perspective).

States are already properly reflected in relation to player's goal, so if you work on them, actions should work the some for both players - there is no difference which player you are using.

`Vec2` is a `pygame.Vector2` type. You can look up it's documentation online. It supports basic vector operations.


### Reset
`reset()` resets the environment: score, all positions and velocities and replay buffer.

### Replays
`showReplay()` displays video of a last game (since the last reset).

### Environment parameters
On top of the code "Constants" section can be found. They can be used to modify the environment.
You can change them to modify the environment:
* `FPS` - number of frames per second - affects the video, but also the simulation (changes number of simulation steps)
* `SCREEN_SIZE` - size of the screen - can't be smaller than pitch size - affects performance of video rendering
* `PITCH_SIZE` - size of the pitch - affects the domain of the problem
* `POST_TOP_Y/POST_BOT_Y` - width of the goal


In [ ]:
env = Haxball()
info = env.info()
print(f"info: {info}\n")

state1, state2, done = env.state()
print(f"state1: {state1}\n")
print(f"state2: {state2}\n")
print(f"done:  {done}")


## Sample game simulation

Below you can see a sample game simulation. Half of the moves are random and half are forward (notice that forward movement is different direction for each player!).

Frames count is specified in `frames` variable. You can change it to see longer games. Length of the game (in seconds) is `frames / FPS`.

After the game, replay is generated.

In [ ]:
import random


def randomVector():
    # Return a random vector from -1 to 1
    return Vec2(random.random() * 2 - 1, random.random() * 2 - 1)


env.reset()

# Play a random game
frames: int = 720

for i in range(frames):
    if random.random() < 0.5:
        # Move randomly
        state1, state2, done = env.step(randomVector(), randomVector())
    else:
        # Move forward
        state1, state2, done = env.step(Vec2(15, 0), Vec2(1, 0))

env.showReplay()

## Movement towards the ball

Below you can see a different game simulation. In this game, players are always moving towards the ball. Notice how the direction towards the ball is calculated from state info.

In [ ]:
import random

env = Haxball()

# Play a random game
frames: int = 1800

for i in range(frames):
    # Move agent towards the ball
    player1State, reward, lastAction = env.getState(1)
    player2State, reward, lastAction = env.getState(2)

    ball1Pos: Vec2 = Vec2(player1State[0], player1State[1])
    player1Pos: Vec2 = Vec2(player1State[4], player1State[5])

    ball2Pos: Vec2 = Vec2(player2State[0], player2State[1])
    player2Pos: Vec2 = Vec2(player2State[4], player2State[5])

    diff1: Vec2 = (ball1Pos - player1Pos).normalize()
    diff2: Vec2 = (ball2Pos - player2Pos).normalize()

    state1, state2, done = env.step(diff1, diff2)

env.showReplay()

# Zadanie wykładowe 2
Stwórz wirtualnego gracza piłki nożnej sterowanego z wykorzystaniem sieci neuronowej. Wykorzystaj jeden z algortmów z wykładu siódmego: DQN, Double DQN lub Dueling DQN.

# Zadania do wykonania
* Opracuj sieć neuronową wyznaczającą wartość Q.
* Zaimplementuj algorytm uczenia sieci neuronowej.
* Naucz piłkarza dryblowania, czyli prowadzenia piłki do wskazanego punktu.
* Naucz piłkarza omijania przeciwnika i prowadzenia piłki do bramki.
* Przeprowadź gry między opracowanymi graczami.



In [2]:
import math
import random
from collections import namedtuple, deque
from pygame.math import Vector2 as Vec2
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [3]:
cos_22dot5 = 1 / 2 * ((2 + (2 ** (1 / 2))) ** (1 / 2))
second_side = 1 - 1 / 4 * (2 + (2 ** (1 / 2)))
possible_moves = [Vec2(0, 0)
    , Vec2(1, 0), Vec2(0, 1), Vec2(-1, 0), Vec2(0, -1)
    , Vec2(1, 1), Vec2(-1, -1), Vec2(-1, 1), Vec2(1, -1)
    , Vec2(cos_22dot5, second_side), Vec2(-cos_22dot5, -second_side), Vec2(-cos_22dot5, second_side),
                  Vec2(cos_22dot5, -second_side)
    , Vec2(second_side, cos_22dot5), Vec2(-second_side, -cos_22dot5), Vec2(-second_side, cos_22dot5),
                  Vec2(second_side, -cos_22dot5)]


In [4]:
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(device)


cpu


In [5]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [6]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [26]:
class TrainingParams:
    def __init__(self, batch_size=128, gamma=0.99, epsilon_start=0.9, epsilon_end=0.02, epsilon_decay=10000,
                 learning_rate=1e-4, update_rate=0.005, replay_memory_size: int = 1e4):
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.update_rate = update_rate
        self.replay_memory_size = replay_memory_size


class RewardData:
    def __init__(self, state, own_goal_post, opp_goal_post):
        self.state = state
        self.own_goal_post = own_goal_post
        self.opp_goal_post = opp_goal_post

    def get_as_array(self):
        return self.state + [self.own_goal_post[0], self.own_goal_post[1], self.own_goal_post[2], self.opp_goal_post[0],
                             self.opp_goal_post[1], self.opp_goal_post[2]]


class Agent:
    def __init__(self, n_observations, n_actions, training_params: TrainingParams, reward_system, add_random_pos: bool,
                 name: str):
        self.n_observations = n_observations
        self.n_actions = n_actions
        self.training_params = training_params

        self.policy_net = DQN(n_observations, n_actions).to(device)
        self.target_net = DQN(n_observations, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.optimizer = optim.AdamW(self.policy_net.parameters(), lr=self.training_params.learning_rate, amsgrad=True)
        self.memory = ReplayMemory(self.training_params.replay_memory_size)
        self.steps_done = 0
        self.reward_system = reward_system
        self.add_random_pos = add_random_pos
        self.name = name

    def select_action(self, state):
        sample = random.random()
        eps_threshold = self.training_params.epsilon_end + (
                self.training_params.epsilon_start - self.training_params.epsilon_end) * \
                        math.exp(-1. * self.steps_done / self.training_params.epsilon_decay)
        self.steps_done += 1
        if sample > eps_threshold:
            with torch.no_grad():
                return self.policy_net(state).max(1).indices.view(1, 1)
        else:
            return torch.tensor([[random.randrange(len(possible_moves))]], device=device)

    def optimize_model(self):
        if len(self.memory) < self.training_params.batch_size:
            return
        transitions = self.memory.sample(self.training_params.batch_size)
        batch = Transition(*zip(*transitions))
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                                batch.next_state)), device=device, dtype=torch.bool)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None]).to(device)
        state_batch = torch.cat(batch.state).to(device)
        action_batch = torch.cat(batch.action).to(device)
        reward_batch = torch.cat(batch.reward).to(device)
        state_action_values = self.policy_net(state_batch).gather(1, action_batch)
        next_state_values = torch.zeros(self.training_params.batch_size, device=device)
        with torch.no_grad():
            next_state_values[non_final_mask] = self.target_net(non_final_next_states).max(2)[0].view(1,self.training_params.batch_size)[0]
        expected_state_action_values = (next_state_values * self.training_params.gamma) + reward_batch

        criterion = nn.SmoothL1Loss()
        loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1)).to(device)

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(self.policy_net.parameters(), 100)
        self.optimizer.step()

    def calculate_reward(self, reward_data: RewardData):
        return self.reward_system(reward_data)

    def save(self, filename):
        torch.save({
            'policy_net': self.policy_net.state_dict(),
            'target_net': self.target_net.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'memory': self.memory,
            'steps_done': self.steps_done
        }, filename)
        print(f"saved to: {filename}")

    def load(self, filename):
        data = torch.load(filename)
        self.policy_net.load_state_dict(data['policy_net'])
        self.target_net.load_state_dict(data['target_net'])
        self.optimizer.load_state_dict(data['optimizer'])
        self.memory = data['memory']
        self.steps_done = data['steps_done']

    def update_networks(self):
        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = self.target_net.state_dict()
        policy_net_state_dict = self.policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key] * self.training_params.update_rate + \
                                         target_net_state_dict[key] * (1 - self.training_params.update_rate)
        self.target_net.load_state_dict(target_net_state_dict)



In [45]:
class Trainer:
    def __init__(self, env: Haxball, agent1: Agent, agent2: Agent, episode_length: int):
        self.env = env
        self.agent1 = agent1
        self.agent2 = agent2
        self.episode_length = episode_length
        self.info = self.env.info()
        self.goal_post1 = (
        self.info["own_postTop_pos"].x, self.info["own_postTop_pos"].y, self.info["own_postBot_pos"].y)
        self.goal_post2 = (
        self.info["opp_postTop_pos"].x, self.info["opp_postTop_pos"].y, self.info["opp_postBot_pos"].y)
        self.team1_goal_post_own = self.goal_post1
        self.team1_goal_post_opp = self.goal_post2
        self.team2_goal_post_own = self.goal_post2
        self.team2_goal_post_opp = self.goal_post1
        self.team1_random_place = None
        self.team2_random_place = None
        self.pitch_size = self.env.info()["pitch_size"]
        self.begin_area_x = 60
        self.begin_area_y = 20
        self.debug = False

    def random_place_on_pitch(self):
        return ((self.pitch_size[0] - self.begin_area_x) * np.random.random_sample() + self.begin_area_x,
                (self.pitch_size[1] - 2 * self.begin_area_y) * np.random.random_sample() + self.begin_area_y)

    def train_players(self, number_of_episodes):
        try:
            episodes_data = []
            print(f"Training for {number_of_episodes} episodes started")
            for _ in tqdm(range(number_of_episodes)):
                time, winning_team = self.one_game()
                episodes_data.append((time, winning_team))
            print(f"Training for {number_of_episodes} episodes finished")
            return episodes_data
        except BaseException:
            print("Training failed, saving data")
            self.save_agents()

    def one_game(self):
        self.env.reset()
        (state1_env, _, _), (state2_env, _, _), done = self.env.state()
        if self.agent1.add_random_pos:
            self.team1_random_place = self.random_place_on_pitch()
            state1_env.append(self.team1_random_place[0])
            state1_env.append(self.team1_random_place[1])
        if self.agent2.add_random_pos:
            self.team2_random_place = self.random_place_on_pitch()
            state2_env.append(self.team2_random_place[0])
            state2_env.append(self.team2_random_place[1])
        for t in range(self.episode_length):
            state1 = RewardData(state1_env, self.team1_goal_post_own, self.team1_goal_post_opp).get_as_array()
            state2 = RewardData(state2_env, self.team2_goal_post_own, self.team2_goal_post_opp).get_as_array()
            step_data_1 = torch.tensor([state1], dtype=torch.float32, device=device)
            step_data_2 = torch.tensor([state2], dtype=torch.float32, device=device)

            action1_index = self.agent1.select_action(step_data_1)
            action2_index = self.agent2.select_action(step_data_2)
            action1 = possible_moves[action1_index.item()]
            action2 = possible_moves[action2_index.item()]
            (next_state1_env, goal_1, _), (next_state2_env, goal_2, _), done = self.env.step(action1, action2)

            if self.agent1.add_random_pos:
                next_state1_env.append(self.team1_random_place[0])
                next_state1_env.append(self.team1_random_place[1])
            if self.agent2.add_random_pos:
                next_state2_env.append(self.team2_random_place[0])
                next_state2_env.append(self.team2_random_place[1])

            next_state1 = RewardData(next_state1_env, self.team1_goal_post_own, self.team1_goal_post_opp)
            next_state2 = RewardData(next_state2_env, self.team2_goal_post_own, self.team2_goal_post_opp)
            next_step_data_1 = torch.tensor([next_state1.get_as_array()], dtype=torch.float32, device=device).unsqueeze(
                0)
            next_step_data_2 = torch.tensor([next_state2.get_as_array()], dtype=torch.float32, device=device).unsqueeze(
                0)
            reward_1 = torch.tensor([[self.agent1.calculate_reward(next_state1)]], dtype=torch.float32, device=device)
            reward_2 = torch.tensor([[self.agent2.calculate_reward(next_state2)]], dtype=torch.float32, device=device)
            if(self.debug):
                print(f"reward1 = {reward_1.item()}; reward2 = {reward_2.item()}")
                print(f"team1_random_place = {self.team1_random_place}; team2_random_place = {self.team2_random_place}")
                print(f"team1_place = {next_state1_env[4]}; team2_place = {next_state1_env[4]}")
                print(f"ball_place = {next_state1_env[0]}; {next_state1_env[1]}")

            # Store the transition in memory
            self.agent1.memory.push(step_data_1, action1_index, next_step_data_1, reward_1)
            self.agent2.memory.push(step_data_2, action2_index, next_step_data_2, reward_2)

            # Move to the next state
            state1_env = next_state1_env
            state2_env = next_state2_env

            # Perform one step of the optimization (on the policy network)
            # print(t)
            self.agent1.optimize_model()
            self.agent2.optimize_model()

            self.agent1.update_networks()
            self.agent2.update_networks()

            if done:
                if goal_1:
                    scoring_player = 1
                else:
                    scoring_player = 2
                return t, scoring_player
        return self.episode_length, 0

    def test_agents(self, test_game_length, with_show_video=False):
        self.env.reset()
        (state1_env, _, _), (state2_env, _, _), done = self.env.state()
        if self.agent1.add_random_pos:
            self.team1_random_place = self.random_place_on_pitch()
            state1_env.append(self.team1_random_place[0])
            state1_env.append(self.team1_random_place[1])
        if self.agent2.add_random_pos:
            self.team2_random_place = self.random_place_on_pitch()
            state2_env.append(self.team2_random_place[0])
            state2_env.append(self.team2_random_place[1])
        for t in range(test_game_length):
            state1 = RewardData(state1_env, self.team1_goal_post_own, self.team1_goal_post_opp).get_as_array()
            state2 = RewardData(state2_env, self.team2_goal_post_own, self.team2_goal_post_opp).get_as_array()
            step_data_1 = torch.tensor([state1], dtype=torch.float32, device=device)
            step_data_2 = torch.tensor([state2], dtype=torch.float32, device=device)

            action1_index = self.agent1.select_action(step_data_1)
            action2_index = self.agent2.select_action(step_data_2)
            action1 = possible_moves[action1_index.item()]
            action2 = possible_moves[action2_index.item()]
            (next_state1_env, goal_1, _), (next_state2_env, goal_2, _), done = self.env.step(action1, action2)

            if self.agent1.add_random_pos:
                next_state1_env.append(self.team1_random_place[0])
                next_state1_env.append(self.team1_random_place[1])
            if self.agent2.add_random_pos:
                next_state2_env.append(self.team2_random_place[0])
                next_state2_env.append(self.team2_random_place[1])

            # Move to the next state
            state1_env = next_state1_env
            state2_env = next_state2_env

            if done:
                break
        self.env.showReplay(custom_string=f"{self.agent1.name}-{self.agent2.name}", with_show_video=with_show_video)

    def save_agents(self):
        print("Saving agents")
        timestamp = datetime.now().strftime('%d-%m-%y--%H-%M-%S')
        self.agent1.save(f"{self.agent1.name}_1_{timestamp}.checkpoint")
        print(f"Saved agent: {self.agent1.name}_1_{timestamp}.checkpoint")
        self.agent2.save(f"{self.agent2.name}_2_{timestamp}.checkpoint")
        print(f"Saved agent: {self.agent2.name}_2_{timestamp}.checkpoint")


In [46]:
def distance(position1, position2):
    return math.sqrt((position1[0] - position2[0]) ** 2 + (position1[1] - position2[1]) ** 2)


def swerving_reward(reward_data: RewardData):  #requires 2 additional arguments in n_observations
    ball_position = (reward_data.state[0], reward_data.state[1])
    place_to_be = (reward_data.state[-2], reward_data.state[-1])
    reward = 400 - distance(ball_position, place_to_be)
    return reward


def swerving_reward_with_penalty(reward_data: RewardData):  #requires 2 additional arguments in n_observations
    ball_position = (reward_data.state[0], reward_data.state[1])
    place_to_be = (reward_data.state[-2], reward_data.state[-1])
    my_position = (reward_data.state[4], reward_data.state[5])
    reward = 400 - 4*distance(ball_position, place_to_be) - 8 * distance(ball_position, my_position)
    return reward

In [41]:
env1 = Haxball()
# Get the number of state observations
(state, _, _), _, _ = env1.state()
n_observations_swerving = len(state) + 8  #goalpost info = 6 + swerving_reward = 2
n_actions_to_select = len(possible_moves)
training_params_swerving1 = TrainingParams(batch_size=128, gamma=0.99, epsilon_start=0.9, epsilon_end=0.02,
                                           epsilon_decay=10000, learning_rate=1e-4, update_rate=0.005,
                                           replay_memory_size=10 ** 6)
training_params_swerving2 = TrainingParams(batch_size=128, gamma=0.99, epsilon_start=0.9, epsilon_end=0.02,
                                           epsilon_decay=10000, learning_rate=1e-4, update_rate=0.005,
                                           replay_memory_size=10 ** 3)
swerving_agent1 = Agent(n_observations_swerving, n_actions_to_select, training_params_swerving1, swerving_reward,
                        add_random_pos=True, name="swerving_agent")
swerving_agent2 = Agent(n_observations_swerving, n_actions_to_select, training_params_swerving2, swerving_reward,
                        add_random_pos=True, name="swerving_agent")

swerving_trainer = Trainer(env=env1, agent1=swerving_agent1, agent2=swerving_agent2, episode_length=1800)

In [11]:
num_of_episodes = 1000

In [12]:
swerving_trainer.train_players(num_of_episodes)
swerving_trainer.save_agents()

Training for 1000 episodes started


/home/student/Documents/GUZW_W2/venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:1040: UserWarning: Using a target size (torch.Size([128, 1, 128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)
100%|██████████████████████████████████████████████████████████████████████████████████| 1000/1000 [3:39:18<00:00, 13.16s/it]


Training for 1000 episodes finished
Saving agents
saved to: swerving_agent_1_17-01-25--07-27-25.checkpoint
Saved agent: swerving_agent_1_17-01-25--07-27-25.checkpoint
saved to: swerving_agent_2_17-01-25--07-27-25.checkpoint
Saved agent: swerving_agent_2_17-01-25--07-27-25.checkpoint


In [25]:
swerving_agent1 = Agent(n_observations_swerving, n_actions_to_select, training_params_swerving1, swerving_reward,
                        add_random_pos=True, name="swerving_agent")
swerving_agent2 = Agent(n_observations_swerving, n_actions_to_select, training_params_swerving2, swerving_reward,
                        add_random_pos=True, name="swerving_agent")
swerving_agent1.load("swerving_agent_1_17-01-25--03-40-52.checkpoint")
swerving_agent2.load("swerving_agent_2_17-01-25--03-40-52.checkpoint")
swerving_trainer.agent1 = swerving_agent1
swerving_trainer.agent2 = swerving_agent2

/tmp/ipykernel_101692/1435209628.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(filename)


In [ ]:
swerving_trainer.test_agents(1800)

In [47]:
env1 = Haxball()
# Get the number of state observations
(state, _, _), _, _ = env1.state()
n_observations_swerving = len(state) + 8  #goalpost info = 6 + swerving_reward = 2
n_actions_to_select = len(possible_moves)
swerving_with_penalty_agent1 = Agent(n_observations_swerving, n_actions_to_select, training_params_swerving1, swerving_reward_with_penalty,
                        add_random_pos=True, name="swerving_agent_with_penalty")
swerving_with_penalty_agent2 = Agent(n_observations_swerving, n_actions_to_select, training_params_swerving2, swerving_reward_with_penalty,
                        add_random_pos=True, name="swerving_agent_with_penalty")

swerving_with_penalty_trainer = Trainer(env=env1, agent1=swerving_with_penalty_agent1, agent2=swerving_with_penalty_agent2, episode_length=1800)
swerving_with_penalty_trainer.debug = True


In [48]:
num_of_episodes = 5

In [49]:
swerving_with_penalty_trainer.train_players(num_of_episodes)
swerving_with_penalty_trainer.save_agents()

Training for 5 episodes started


  0%|          | 0/5 [00:00<?, ?it/s]

reward1 = -715.9512939453125; reward2 = -706.7108764648438
team1_random_place = (555.8391909830223, 370.2365061288654); team2_random_place = (573.3373964733561, 328.538891639004)
team1_place = 169.35476506216727; team2_place = 169.35476506216727
ball_place = 320.0; 220.0
reward1 = -713.4442138671875; reward2 = -699.3170166015625
team1_random_place = (555.8391909830223, 370.2365061288654); team2_random_place = (573.3373964733561, 328.538891639004)
team1_place = 169.72369398646828; team2_place = 169.72369398646828
ball_place = 320.0; 220.0
reward1 = -713.3610229492188; reward2 = -693.217529296875
team1_random_place = (555.8391909830223, 370.2365061288654); team2_random_place = (573.3373964733561, 328.538891639004)
team1_place = 170.0280603490166; team2_place = 170.0280603490166
ball_place = 320.0; 220.0
reward1 = -709.2078247070312; reward2 = -692.541748046875
team1_random_place = (555.8391909830223, 370.2365061288654); team2_random_place = (573.3373964733561, 328.538891639004)
team1_pla

 20%|██        | 1/5 [00:25<01:41, 25.30s/it]

reward1 = -1183.0057373046875; reward2 = -1318.6607666015625
team1_random_place = (555.8391909830223, 370.2365061288654); team2_random_place = (573.3373964733561, 328.538891639004)
team1_place = 528.3087764867139; team2_place = 528.3087764867139
ball_place = 322.5198098267455; 225.59170290996514
reward1 = -1176.9312744140625; reward2 = -1309.6304931640625
team1_random_place = (555.8391909830223, 370.2365061288654); team2_random_place = (573.3373964733561, 328.538891639004)
team1_place = 527.622034728714; team2_place = 527.622034728714
ball_place = 322.5198098267455; 225.59170290996514
reward1 = -1174.1385498046875; reward2 = -1299.822265625
team1_random_place = (555.8391909830223, 370.2365061288654); team2_random_place = (573.3373964733561, 328.538891639004)
team1_place = 526.9126135934948; team2_place = 526.9126135934948
ball_place = 322.5198098267455; 225.59170290996514
reward1 = -1171.3077392578125; reward2 = -1295.6553955078125
team1_random_place = (555.8391909830223, 370.236506128

...

 80%|████████  | 4/5 [01:31<00:23, 23.39s/it]

reward1 = -1537.2254638671875; reward2 = -1148.251220703125
team1_random_place = (424.4622434232347, 270.13718065217904); team2_random_place = (405.8243900414065, 72.91289570878605)
team1_place = 585.5996261944348; team2_place = 585.5996261944348
ball_place = 280.72716185795343; 211.00079048764016
reward1 = -1535.7965087890625; reward2 = -1148.336181640625
team1_random_place = (424.4622434232347, 270.13718065217904); team2_random_place = (405.8243900414065, 72.91289570878605)
team1_place = 585.4810174406377; team2_place = 585.4810174406377
ball_place = 280.72716185795343; 211.00079048764016
reward1 = -1536.13818359375; reward2 = -1144.5220947265625
team1_random_place = (424.4622434232347, 270.13718065217904); team2_random_place = (405.8243900414065, 72.91289570878605)
team1_place = 585.3831652187552; team2_place = 585.3831652187552
ball_place = 280.72716185795343; 211.00079048764016
reward1 = -1535.1654052734375; reward2 = -1139.9508056640625
team1_random_place = (424.4622434232347, 27

100%|██████████| 5/5 [01:51<00:00, 22.34s/it]

reward1 = -1166.900146484375; reward2 = 117.93470764160156
team1_random_place = (208.61053429207396, 176.472616563008); team2_random_place = (376.19781574235265, 43.69119369970517)
team1_place = 58.44935731106272; team2_place = 58.44935731106272
ball_place = 320.0; 220.0
reward1 = -1175.5867919921875; reward2 = 111.14466094970703
team1_random_place = (208.61053429207396, 176.472616563008); team2_random_place = (376.19781574235265, 43.69119369970517)
team1_place = 57.93346493916499; team2_place = 57.93346493916499
ball_place = 320.0; 220.0
reward1 = -1186.9229736328125; reward2 = 101.00696563720703
team1_random_place = (208.61053429207396, 176.472616563008); team2_random_place = (376.19781574235265, 43.69119369970517)
team1_place = 56.595353732349366; team2_place = 56.595353732349366
ball_place = 320.0; 220.0
reward1 = -1198.34326171875; reward2 = 87.86515808105469
team1_random_place = (208.61053429207396, 176.472616563008); team2_random_place = (376.19781574235265, 43.69119369970517)
t

saved to: swerving_agent_with_penalty_1_17-01-25--12-05-57.checkpoint
Saved agent: swerving_agent_with_penalty_1_17-01-25--12-05-57.checkpoint
saved to: swerving_agent_with_penalty_2_17-01-25--12-05-57.checkpoint
Saved agent: swerving_agent_with_penalty_2_17-01-25--12-05-57.checkpoint


In [34]:
swerving_with_penalty_trainer.test_agents(1800)


tmp_swerving_agent-swerving_agent_17-01-25--11-53-02.mp4
